In [1]:
def countLines():
    with open('C:\\Users\\liuhw3\\Downloads\\code\\data\\ads.txt', 'r', encoding='utf-8') as f:
        print("ads.txt 行数：")
        print(len(f.readlines()))
        
    with open('C:\\Users\\liuhw3\\Downloads\\code\\data\\nads.txt', 'r', encoding='utf-8') as f:
        print("nads.txt 行数：")
        print(len(f.readlines()))

countLines()

ads.txt 行数：
284433
nads.txt 行数：
293126


# 0.txt2csv

In [5]:
import re
import pandas as pd

In [6]:
def ads2csv():
    contexts = []
    with open('C:\\Users\\liuhw3\\Downloads\\code\\data\\ads.txt', 'r', encoding='utf-8') as f:    
        line = f.readline() 
        pattern = re.compile("\"content\"\:\"([\u4e00-\u9fa5]|\S|\s)*\"")
        while line:   
            # print(line)
            result = pattern.search(line)
            result = re.sub('\<(.*?)\>', '', result.group(0))
            contexts.append(result[11:-1])
            line = f.readline()
    
    ads = {}
    ads['context'] = contexts
    ads["tag"] = [1 for i in range(len(contexts))]

    df = pd.DataFrame(ads)
    df.to_csv("./data/ads.csv", index=False, encoding='utf-8')
    
def nads2csv():
    contexts = []
    with open('C:\\Users\\liuhw3\\Downloads\\code\\data\\nads.txt', 'r', encoding='utf-8') as f:   
        line = f.readline() 
        while line:   
            # print(line)
            tmp = re.sub('http(.*?)(jpg|jpeg|png|gif)', '', line)
            result = re.sub('\\r\\n', '', tmp)
            contexts.append(result[:-6])
            line = f.readline()
    
    nads = {}
    nads['context'] = contexts
    nads["tag"] = [0 for i in range(len(contexts))]

    df = pd.DataFrame(nads)
    df.to_csv("./data/nads.csv", index=False, encoding='utf-8')

In [7]:
def txt2csv():
    ads2csv()
    nads2csv()
    
txt2csv()

# 1.getdata

In [9]:
ads = pd.read_csv("./data/ads.csv")
nads = pd.read_csv("./data/nads.csv")

In [10]:
odata = pd.concat([ads, nads], axis=0, ignore_index=True)

In [11]:
odata.head()

,context,tag
0,原标题:高能预警！玉溪这里一大波岗位正在向你靠近，忍不住跳槽！【招聘信息】店长 文案 哺乳指...,1
1,现如今智能设备早已成为了优质生活的一部分，如今像小米、360、华为、天猫等大厂都纷纷推出了自...,1
2,原标题:用它刷牙，10多年黄牙口臭全没了，牙齿竟还了白一度 乐活甄选馆 点击发现更多好物 ...,1
3,文章中，我将总结新老Python程序员常犯的一些错误，以帮助你们在自己的工作避免犯同样或类似...,1
4,在这个“颜值即正义”的时代，许多在选择一款手机的时候，不仅要求手机拥有时尚的外观设计，同时还...,1


In [12]:
data = odata.sample(frac=1).reset_index(drop=True)
# data = data.reindex(np.random.permutation(data.index))

In [13]:
data.head()

,context,tag
0,【e公司直播】格力电器董事会换届 董明珠是否连任董事？ e公司讯，1月16日下午2：30，格...,0
1,美拍视频: 吃饭饭美拍视频,0
2,原标题:7月1日起，你的养老金将有重大调整！新疆人注意，关乎你的钱袋子！点击阿克苏我的家关注...,1
3,虽说今年新iPhone的表现不是那么惊世骇俗，但亮点也是有的，譬如加入了“双卡双待”豪华套餐...,1
4,原标题:Happy Jazz Year | 在爵士跨年夜中，迎来2019！ ...,1


In [14]:
data.shape

(577559, 2)

In [15]:
data = data.dropna(axis=0, how='any')

In [16]:
data.shape

(577224, 2)

# 2.process

In [17]:
import jieba

In [18]:
stopwords = []
with open('./data/stopwords.txt', "r", encoding='utf-8') as f:
    line = f.readline()
    while line:
        stopwords.append(line[:-1])
        line = f.readline()
stopwords = set(stopwords)

In [19]:
def splitWord(text):
    seg_list = list(jieba.cut(text))
    words = [w for w in seg_list if w not in stopwords]
    words = " ".join(words)
    return words

In [35]:
data['words'] = data['context'].apply(splitWord)

In [36]:
data.head()

,context,tag,words
0,【e公司直播】格力电器董事会换届 董明珠是否连任董事？ e公司讯，1月16日下午2：30，格...,0,e 公司 直播 格力电器 董事会 换届 董明珠 连任 董事 e 公司 讯 格力电器 迎来 2...
1,美拍视频: 吃饭饭美拍视频,0,美拍 视频 吃饭 饭美 拍 视频
2,原标题:7月1日起，你的养老金将有重大调整！新疆人注意，关乎你的钱袋子！点击阿克苏我的家关注...,1,原 标题 日起 养老金 调整 新疆 关乎 钱袋子 点击 阿克苏 家 关注 刚刚 国务院 印发...
3,虽说今年新iPhone的表现不是那么惊世骇俗，但亮点也是有的，譬如加入了“双卡双待”豪华套餐...,1,新 iPhone 表现 惊世骇俗 亮点 双卡 双待 豪华 套餐 行列 算是 中国国情 考量 ...
4,原标题:Happy Jazz Year | 在爵士跨年夜中，迎来2019！ ...,1,原 标题 Happy Jazz Year 爵士 跨年 夜中 迎来 2019


In [42]:
data.to_csv("./data/data.csv", index=False, encoding='utf-8')